In [130]:
from datetime import datetime
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from researcher_impact.plotting import save_plot

In [69]:
pio.templates.default = "plotly_white"

In [48]:
result_file_location = 'results/'
os.makedirs(result_file_location, exist_ok=True)

In [5]:
company_aliases = {
    "Google": "Google",
    "Google Research": "Google",
    "Google Brain": "Google",
    "Google Inc.": "Google",
    "Google AI, Brain team": "Google",
    "Google Research, Brain Team": "Google",
    "Google AI": "Google",
    "Google Brain,Google Research": "Google",
    "Google Inc": "Google",
    "DeepMind": "DeepMind",
    "Google DeepMind": "DeepMind",
    "Meta AI": "Meta",
    "MetaAI": "Meta",
    "Facebook AI Research": "Meta",
    "Facebook AI research": "Meta",
    "Facebook": "Meta",
    "Facebook AI": "Meta",
    "OpenAI": "OpenAI",
    "Open AI": "OpenAI",
    "Microsoft Research": "Microsoft",
    "Microsoft": "Microsoft",
    "Microsoft Research,Peking University": "Microsoft",
    "Microsoft Bing": "Microsoft",
    "Alibaba Group": "Alibaba",
    "NVIDIA": "NVIDIA",
    "Nvidia": "NVIDIA",
    "Baidu Research- Silicon Valley AI Lab": "Baidu",
    "Baidu": "Baidu",
    "Amazon": "Amazon",
}

- Sort by publication date
- Filter companies of interest + rename companies to consistent alias => new DataFrame
- Get the maximum envelope of the data => new DataFrame
- Add on the last maximum to the current date, so there's a horizontal line continuing until the current date

In [170]:
# Download dataset from the Parameters, Compute and Data Trends in ML sheet
sheet_id = '1AAIebjNsnJj_uKALHbXNfn3_YsT6sHXtCU0q7OIPuc4'
data_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet='
df = pd.read_csv(data_url + 'ALL%20ML%20SYSTEMS')

In [171]:
cols_of_interest = ["System", "Organization", "Publication date", "Training compute (FLOP)"]

In [172]:
df = df.dropna(subset=["Organization", "Training compute (FLOP)"])

In [173]:
# Rejected samples
# We don't think the full training run was actually done
df.drop(df[df["System"] == "Megatron-LM (1T)"].index, inplace=True)
# Kingma was affiliated with OpenAI on the paper for this system, but only in the 2017 version rather than the original 2014 version
df.drop(df[df["System"] == "ADAM (CIFAR-10)"].index, inplace=True)



In [174]:
df['Publication date'] = pd.to_datetime(df['Publication date'])  # Ensure date column is in datetime format

In [175]:
df.sort_values('Publication date', inplace=True)
df.reset_index(inplace=True)

In [176]:
df[cols_of_interest]

,System,Organization,Publication date,Training compute (FLOP)
0,Theseus,Bell Laboratories,1950-07-02,4.000000e+01
1,Perceptron Mark I,Cornell Aeronautical Laboratory,1957-01-01,6.950000e+05
2,Pandemonium (morse),Massachusetts Institute of Technology,1959-02-01,6.000000e+08
3,Samuel Neural Checkers,IBM,1959-07-01,4.280000e+08
4,ADALINE,Stanford University,1960-06-30,9.900000e+03
...,...,...,...,...
171,BLOOM,"Hugging Face, BigScience",2022-11-08,1.800000e+23
172,AR-LDM,"Alibaba,University of Waterloo,Vector Institute",2022-11-20,5.100000e+20
173,LLaMA (65B),Meta AI,2023-02-24,5.500000e+23
174,GPT-4,OpenAI,2023-03-15,2.100000e+25


## Filter companies of interest + rename companies to consistent alias

In [177]:
rows = []
for i, row in df.iterrows():
    orgs = row["Organization"]
    print(orgs)
    org_list = [org.strip() for org in orgs.split(",")]
    for org in org_list:
        if org in company_aliases.keys():
            alias = company_aliases[org]
            new_row = row.copy()
            new_row["Organization"] = alias
            rows.append(new_row)

company_df = pd.DataFrame(rows)
company_df[cols_of_interest]

Bell Laboratories
Cornell Aeronautical Laboratory
Massachusetts Institute of Technology
IBM
Stanford University
NHK Broadcasting Science Research Laboratories
University of California
Princeton University
Stanford, CalTech
AT&T Bell Laboratories
Carnegie Mellon University 
IBM
Indian Statistical Institute
Carnegie Mellon University
The Technical University of Munich
National Chiao Tung University
AT&T Labs
Mitsubishi Electric Research Labs and Compaq CRL
Université de Montréal
IDSIA and TU Munich
Stanford
IDSIA ; University of Lugano & SUPSI
University of Montreal
Brno University of Technology, Johns Hopkins University
Brno University of Technology, Johns Hopkins University
IDSIA
University of Toronto
University of Toronto
University of Toronto
Google
University of Toronto
Universidad Nacional de Cordoba,Xerox Research Centre Europe,Inteligent Systems Lab Amsterdam,University of Amsterdam,LEAR Team,INRIA Grenoble
IDSIA
Google
NYU
CNRS, Google
DeepMind
Univeristy of Amsterdam
Universite

,System,Organization,Publication date,Training compute (FLOP)
29,Unsupervised High-level Feature Learner,Google,2012-07-12,6.000000e+17
33,Word2Vec (large),Google,2013-10-16,3.890000e+16
35,TransE,Google,2013-12-05,1.340000e+18
36,DQN,DeepMind,2013-12-19,2.300000e+15
39,SPPNet,Microsoft,2014-06-18,3.410000e+18
...,...,...,...,...
168,AlexaTM 20B,Amazon,2022-08-02,2.040000e+23
170,Whisper,OpenAI,2022-09-21,4.650000e+22
173,LLaMA (65B),Meta,2023-02-24,5.500000e+23
174,GPT-4,OpenAI,2023-03-15,2.100000e+25


In [178]:
company_max_compute_df = company_df.copy()

In [179]:
company_max_compute_df["Training compute (FLOP)"] = company_df.groupby("Organization")['Training compute (FLOP)'].cummax()
company_max_compute_df[cols_of_interest]

,System,Organization,Publication date,Training compute (FLOP)
29,Unsupervised High-level Feature Learner,Google,2012-07-12,6.000000e+17
33,Word2Vec (large),Google,2013-10-16,6.000000e+17
35,TransE,Google,2013-12-05,1.340000e+18
36,DQN,DeepMind,2013-12-19,2.300000e+15
39,SPPNet,Microsoft,2014-06-18,3.410000e+18
...,...,...,...,...
168,AlexaTM 20B,Amazon,2022-08-02,2.040000e+23
170,Whisper,OpenAI,2022-09-21,3.140000e+23
173,LLaMA (65B),Meta,2023-02-24,5.500000e+23
174,GPT-4,OpenAI,2023-03-15,2.100000e+25


In [180]:
rows_to_add = []
for org, group_data in company_max_compute_df.groupby("Organization"):
    print(org, group_data["Training compute (FLOP)"].max())
    current_date = datetime.now().date()  # Get the current date
    compute = group_data["Training compute (FLOP)"].max()
    # Create a new row with NaN for all columns except "Organization", "Publication date", and "Training compute (FLOP)"
    new_row = pd.Series(
        {
            "Organization": org,
            "Publication date": current_date,
            "Training compute (FLOP)": compute,
        }
    )
    rows_to_add.append(new_row)

# Create a DataFrame from the rows to be added
new_data = pd.DataFrame(rows_to_add)

# Concatenate the new_data DataFrame to the original filtered_df
company_max_compute_df = pd.concat([company_max_compute_df, new_data], ignore_index=True)

Alibaba 3.6e+22
Amazon 2.04e+23
Baidu 3.14e+23
DeepMind 6.31e+23
Google 7.34e+24
Meta 5.5e+23
Microsoft 1.35e+24
NVIDIA 1.35e+24
OpenAI 2.1e+25


In [181]:
company_max_compute_df

,index,System,Domain,Task,Organization,Organization Categorization,Authors,Publication date,Reference,Link,...,Training time notes,Training hardware,Approach,Training compute cost (2020 USD),Compute cost notes,Self-supervised training,Compute Sponsor Categorization,Epistemic status,Abstract,Last Modified
0,374.0,Unsupervised High-level Feature Learner,Vision,Image classification,Google,Industry,"Quoc V. Le, Marc'Aurelio Ranzato, Rajat Monga,...",2012-07-12 00:00:00,Building High-level Features Using Large Scale...,https://arxiv.org/pdf/1112.6209.pdf,...,"""We train this network using model parallelism...",NaN,Unsupervised,NaN,Hardware not reported,NaN,Industry,Likely,We consider the problem of building high-level...,2023-06-15 15:50:10
1,359.0,Word2Vec (large),Language,Semantic embedding,Google,Industry,"T Mikolov, I Sutskever, K Chen, GS Corrado",2013-10-16 00:00:00,Distributed Representations of Words and Phras...,https://arxiv.org/abs/1310.4546,...,NaN,NaN,NaN,0.55,NaN,NaN,Industry,NaN,NaN,2023-06-14 14:54:39
2,355.0,TransE,Other,Entity embedding,Google,Industry - Academia Collaboration,"Antoine Bordes, Nicolas Usunier, Alberto Garci...",2013-12-05 00:00:00,Translating Embeddings for Modeling Multi- rel...,https://papers.nips.cc/paper/2013/hash/1cecc7a...,...,NaN,NaN,NaN,17.58,NaN,NaN,Industry,NaN,NaN,2023-05-29 20:51:04
3,352.0,DQN,Games,Atari,DeepMind,Industry,"V Mnih, K Kavukcuoglu, D Silver, A Graves",2013-12-19 00:00:00,Playing Atari with Deep Reinforcement Learning,https://arxiv.org/abs/1312.5602,...,NaN,NaN,NaN,0.04,NaN,NaN,Industry,NaN,NaN,2023-06-14 14:57:17
4,341.0,SPPNet,Vision,Image classification,Microsoft,Industry - Academia Collaboration,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",2014-06-18 00:00:00,Spatial Pyramid Pooling in Deep Convolutional ...,https://arxiv.org/abs/1406.4729,...,"""All networks in this paper can be trained on ...",NVIDIA GeForce GTX TITAN,NaN,65.07,NaN,NaN,Industry,NaN,NaN,2023-08-01 10:30:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,NaN,NaN,NaN,NaN,Google,NaN,NaN,2023-08-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,NaN,NaN,NaN,NaN,Meta,NaN,NaN,2023-08-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,Microsoft,NaN,NaN,2023-08-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,NaN,NaN,NaN,NaN,NVIDIA,NaN,NaN,2023-08-03,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
company_max_compute_df[cols_of_interest]

,System,Organization,Publication date,Training compute (FLOP)
0,Unsupervised High-level Feature Learner,Google,2012-07-12 00:00:00,6.000000e+17
1,Word2Vec (large),Google,2013-10-16 00:00:00,6.000000e+17
2,TransE,Google,2013-12-05 00:00:00,1.340000e+18
3,DQN,DeepMind,2013-12-19 00:00:00,2.300000e+15
4,SPPNet,Microsoft,2014-06-18 00:00:00,3.410000e+18
...,...,...,...,...
104,NaN,Google,2023-08-03,7.340000e+24
105,NaN,Meta,2023-08-03,5.500000e+23
106,NaN,Microsoft,2023-08-03,1.350000e+24
107,NaN,NVIDIA,2023-08-03,1.350000e+24


In [183]:
current_max = 0
current_max_org = "Google"
global_max_rows = []
for i, row in company_max_compute_df.iterrows():
    if row["Training compute (FLOP)"] > current_max:
        current_max = row["Training compute (FLOP)"]
        if row["Organization"] != current_max_org:
            current_max_org = row["Organization"]
            global_max_rows.append(row)
global_max_df = pd.DataFrame(global_max_rows)
global_max_df[cols_of_interest]

,System,Organization,Publication date,Training compute (FLOP)
4,SPPNet,Microsoft,2014-06-18,3.410000e+18
5,Seq2Seq LSTM,Google,2014-09-10,7.300000e+18
6,"MSRA (C, PReLU)",Microsoft,2015-02-06,2.400000e+19
8,AlphaGo Fan,DeepMind,2015-10-01,3.800000e+20
12,GNMT,Google,2016-09-26,6.900000e+21
15,AlphaGo Master,DeepMind,2017-01-01,1.500000e+23
78,Megatron-Turing NLG 530B,Microsoft,2021-10-11,1.350000e+24
87,PaLM (540B),Google,2022-04-04,2.530000e+24
98,GPT-4,OpenAI,2023-03-15,2.100000e+25


In [234]:
# Create a line plot
fig = px.line(
    company_max_compute_df[company_max_compute_df["Organization"].isin(["Google", "DeepMind", "OpenAI", "Meta", "Microsoft"])],
    x="Publication date", 
    y="Training compute (FLOP)", 
    line_shape="hv",
    color="Organization",
    labels={"Training compute (FLOP)": "Largest published training run to date (FLOP)"},
    category_orders={"Organization": ["OpenAI", "Google", "Microsoft", "DeepMind", "Meta"]},
    hover_data=["System"],
)
fig.add_trace(
    go.Scatter(
        x=global_max_df["Publication date"],
        y=global_max_df["Training compute (FLOP)"],
        text=global_max_df["System"],
        textposition="top left",
        line={"color": "black"},
        mode="markers+text",
        name="Changes in leader",
    )
)
fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(["2020-05-28"]),
        y=[3.14e23],
        text=["GPT-3 (175B)"],
        textposition="top left",
        line={"color": "black"},
        mode="markers+text",
        marker_symbol="x",
        name="Other notable systems",
    )
)
# fig.add_trace(
#     go.Scatter(
#         x=pd.to_datetime(["2020-05-28", "2023-02-24"]),
#         y=[3.14e23, 5.50e23],
#         text=["GPT-3 (175B)", "LLaMA (65B)"],
#         textposition=["top left", "middle left"],
#         line={"color": "black"},
#         mode="markers+text",
#         marker_symbol="x",
#         name="Other notable systems",
#     )
# )

# Convert year to datetime
fig.update_yaxes(type="log")
# Show all years on x axis
fig.update_xaxes(nticks=12)
# Make sure the labels are fully shown
fig.update_xaxes(range=[pd.to_datetime(2011, format="%Y"), datetime.now().date()])
fig.update_layout(
    legend=dict(
        title="",
        orientation="h",
        # yanchor="top",
        y=-0.15,
        # xanchor="center",
        x=0,
    ),
)
# Edit figure layout
fig.update_layout(
    autosize=False,
    width=400,
    height=415,
    font=dict(size=10),
    margin=dict(l=20, r=20, t=20, b=0),
)

save_plot(fig, result_file_location, 'companies_largest_compute_all')

fig.show()